# Copy Number Pipeline

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np

from src.CCLE_postp_function import *
from genepy import terra
from genepy.utils import helper as h
from genepy.google import gcp
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm
from genepy.google.google_sheet import dfToSheet

from scipy.stats import pearsonr,spearmanr

from bokeh.plotting import *
from IPython.display import Image,display
import seaborn as sns

from biomart import BiomartServer
import io

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Loading BokehJS ...

## boot up

- you first need to go to [taiga](https://cds.team/taiga/dataset) and create some new datasets for the virtual release
- the easiest way to create a new dataset is to upload an empty file (since at least one file is required). This empty file can be deleted when you update the dataset with a new version

we are instanciating all the parameters needed for this pipeline to run

In [3]:
samplesetname = "21Q1"

refworkspace="broad-firecloud-ccle/DepMap_WES_CN_hg38"
genelist_hg38 = 'ftp://ftp.ncbi.nlm.nih.gov/pub/CCDS/current_human/CCDS.20180614.txt'

refsheet_url = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
potential_list_url = "https://docs.google.com/spreadsheets/d/1BEgH03V4OmGhYeciLCZV00h6hp3WkO0basahS93akCE"

CNWESmethods = [
    "gatk/PreProcessingForVariantDiscovery_GATK4/8",
    "GP-TAG/Manta_SomaticSV/9",
    "gkugener/ArrayOfFilesToTxt/1",
    "vdauwera/BamToUnmappedRGBams/4",
    "gatk/CNV_Somatic_Pair_Workflow/9",
    "gkugener/Aggregate_CN_seg_files/2"
]

#version 102
ensemblserver = "http://nov2020.archive.ensembl.org/biomart" 

In [67]:
release = samplesetname
# we initialize the workspaces manager from dalmatian
refwm = dm.WorkspaceManager(refworkspace)

potential_list = sheets.get(potential_list_url).sheets[0].to_frame().values.T[0].tolist()

ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

len(ccle_refsamples)

6771

In [105]:
## missing tissue annot
set(ccle_refsamples[ccle_refsamples['primary_site'].isna() | ccle_refsamples['primary_disease'].isna() | ccle_refsamples['subtype'].isna() | ccle_refsamples['origin'].isna()].arxspan_id

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-55CRsR,ACH-001182,1,NaN,NaN,hybrid_capture,1.793164e+09,NaN,RPMI6666,PT-oFgMemwT,CVCL_1665,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
CDS-c7bFKt,ACH-000471,1,NaN,NaN,hybrid_capture,6.444237e+09,NaN,LI7,PT-vruNvPAR,CVCL_3840,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
CDS-5hu4Rw,ACH-001187,1,NaN,NaN,hybrid_capture,2.540050e+09,NaN,SF8657,PT-rocMG2rE,CVCL_K085,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
CDS-VjQVjt,ACH-001182,1,NaN,NaN,raindance,1.969057e+09,NaN,RPMI6666,PT-oFgMemwT,CVCL_1665,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
CDS-EclxvG,ACH-000471,1,NaN,NaN,raindance,1.504851e+09,NaN,LI7,PT-vruNvPAR,CVCL_3840,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-J80Eb1,ACH-002034,1,NaN,PDO-21298,rna,4.940856e+09,NaN,T3M11,PT-hYmRJ4Oj,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-Hb7hHv,ACH-002080,1,NaN,PDO-21298,rna,4.613839e+09,NaN,TN2,PT-h2eeF6an,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-3VSkPT,ACH-002687,1,NaN,PDO-21569,rna,5.219565e+09,NaN,WM3772F,PT-MJutykcs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-2FgC1U,ACH-001707,1,NaN,"PDO-20722,PDO-21642",rna,7.056679e+09,NaN,WAOSEL,PT-y3RbI7uD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN


In [106]:
## missing name/arxspan
ccle_refsamples[ccle_refsamples['arxspan_id'].isna() | ccle_refsamples['stripped_cell_name'].isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [107]:
# clean annots:
set(ccle_refsamples.primary_disease)

{'Adrenal Cancer',
 'Bile Duct Cancer',
 'Bladder Cancer',
 'Bone Cancer',
 'Brain Cancer',
 'Breast Cancer',
 'Cervical Cancer',
 'Colon/Colorectal Cancer',
 'Embryonal Cancer',
 'Endometrial/Uterine Cancer',
 'Engineered',
 'Esophageal Cancer',
 'Eye Cancer',
 'Gallbladder Cancer',
 'Gastric Cancer',
 'Head and Neck Cancer',
 'Kidney Cancer',
 'Leukemia',
 'Liposarcoma',
 'Liver Cancer',
 'Lung Cancer',
 'Lymphoma',
 'Myeloma',
 'Neuroblastoma',
 'Ovarian Cancer',
 'Pancreatic Cancer',
 'Prostate Cancer',
 'Rhabdoid',
 'Sarcoma',
 'Skin Cancer',
 'Teratoma',
 'Thymic Cancer',
 'Thyroid Cancer',
 'Unknown',
 'Vulvar',
 'esophagus_squamous',
 nan,
 'normal'}

In [108]:
set(ccle_refsamples.primary_site)

{'Colon',
 'Embryonal',
 'Esophageal Cancer',
 'Placenta',
 'Thymus',
 'abdomen',
 'ascites',
 'autonomic_ganglia',
 'biliary_tract',
 'bone',
 'bone_marrow',
 'brain',
 'breast',
 'central_nervous_system',
 'cervix',
 'endometrium',
 'engineered',
 'eye',
 'fibroblast',
 'haematopoietic_and_lymphoid_tissue',
 'kidney',
 'large_intestine',
 'liver',
 'lung',
 'lymph_node',
 nan,
 'oesophagus',
 'ovary',
 'pancreas',
 'pericardial_effusion',
 'pleura',
 'pleural_effusion',
 'prostate',
 'salivary_gland',
 'skin',
 'small_intestine',
 'soft_tissue',
 'spleen',
 'stomach',
 'thyroid',
 'upper_aerodigestive_tract',
 'urinary_tract',
 'uvea',
 'vul'}

In [109]:
set(ccle_refsamples.age)

{'0',
 '0.1',
 '0.25',
 '0.33',
 '0.5',
 '0.75',
 '0.8',
 '1',
 '1.4',
 '1.6',
 '1.67',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '2.6',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '3.5',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '92',
 'Adult',
 'C',
 'Children',
 'Embryo',
 'Fetus',
 'U',
 nan}

In [112]:
ccle_refsamples[ccle_refsamples.age.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-yYeDem,ACH-001429,2,NaN,PDO-21569,rna,6.136561e+09,NaN,CCLFPEDS0009T,PT-gpiijlSj,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-OT217E,ACH-001338,1,NaN,PDO-21298,rna,5.395430e+09,NaN,CHP134,PT-sqRdd0QF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-a8BCnW,ACH-001996,1,NaN,PDO-21298,rna,5.692887e+09,NaN,ICC18,PT-fZWks7bl,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-zeFugQ,ACH-002707,1,NaN,PDO-21569,rna,4.253798e+09,NaN,IMSM2,PT-KQHkzAJs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-GOI63U,ACH-002077,1,NaN,PDO-21298,rna,4.736551e+09,NaN,LU165,PT-3za4R2YQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-ldvG6N,ACH-001553,1,NaN,PDO-21298,rna,4.695594e+09,NaN,MEGA2,PT-SMrpbPt7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-LFK9QA,ACH-002013,1,NaN,PDO-21298,rna,4.680192e+09,NaN,MM28,PT-ws3gMJ8I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-SdYhEt,ACH-002084,1,NaN,PDO-21298,rna,5.914711e+09,NaN,MMAC,PT-ViFHoQt0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-gyyVs5,ACH-001992,1,NaN,PDO-21298,rna,6.628526e+09,NaN,ONE58,PT-77bcwWNt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-IgYH22,ACH-001633,1,NaN,PDO-21298,rna,5.966996e+09,NaN,PEO6,PT-kjmIMuN4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN


In [115]:
set(ccle_refsamples.subtype)

{'ALL',
 'AML',
 'ATL',
 'ATRT',
 'Acute Lymphoblastic Leukemia (ALL), B-cell',
 'Adenocarcinoma, clear cell',
 'B-cell',
 'B-cell, Non-Hodgkins, Burkitts',
 'Breast Ductal Carcinoma',
 'CLL',
 'CML',
 'Carcinoma',
 'Cholangiocarcinoma',
 'Engineered',
 'Ewing sarcoma',
 'Ewing_sarcoma',
 'Ewings Sarcoma',
 'Glioblastoma',
 'MMMT',
 'MPNST',
 'Medulloblastoma',
 'Melanoma',
 'Mesothelioma',
 'Mixed Uterine Adenocarcinoma and Rhabdomyosarcoma',
 'NSCLC',
 'Neuroblastoma',
 'Ovarian carcinoma',
 'PNET',
 'RMS',
 'SCCOHT',
 'SCLC',
 'Small Cell Lung Cancer (SCLC)',
 'Squamous Cell Carcinoma',
 'Testicular',
 'Uveal Melanoma',
 'acute_biphenotypic_leukemia',
 'adrenal_carcinoma',
 'bladder_carcinoma',
 'breast_adenocarcinoma',
 'breast_carcinoma',
 'breast_ductal_carcinoma',
 'brenner_tumor',
 'caecum_adenocarcinoma',
 'cervical_adenocarcinoma',
 'cervical_carcinoma',
 'cervical_squamous',
 'cholangiocarcinoma',
 'chondrosarcoma',
 'chordoma',
 'choriocarcinoma',
 'clear_cell_carcinoma',
 

In [119]:
set(ccle_refsamples.origin)

{'adrenal_cortex',
 'bile_duct',
 'bladder',
 'blood',
 'bone',
 'breast',
 'central_nervous_system',
 'cervix',
 'colorectal',
 'embryo',
 'engineered',
 'engineered_blood',
 'engineered_breast',
 'engineered_central_nervous_system',
 'engineered_kidney',
 'engineered_lung',
 'engineered_ovary',
 'engineered_prostate',
 'engineered_soft_tissue',
 'esophagus',
 'eye',
 'fibroblast',
 'gastric',
 'kidney',
 'liver',
 'lung',
 'lymphocyte',
 nan,
 'ovary',
 'pancreas',
 'peripheral_nervous_system',
 'plasma_cell',
 'prostate',
 'skin',
 'soft_tissue',
 'thyroid',
 'unknown',
 'upper_aerodigestive',
 'urinary_tract',
 'uterus'}

In [118]:
set(ccle_refsamples.columns)

{'18q1',
 '18q2',
 '18q3',
 '18q4',
 '19q1',
 '19q2',
 '19q3',
 '19q4',
 '20q',
 '20q1',
 '20q2',
 '20q4',
 '21Q1',
 'PDO',
 'age',
 'arxspan_id',
 'baits',
 'bam_public_sra_path',
 'bam_qc',
 'ccle_name',
 'cellosaurus_id',
 'comments',
 'condition',
 'crc32c_hash',
 'datatype',
 'internal_bai_filepath',
 'internal_bam_filepath',
 'legacy_bai_filepath',
 'legacy_bam_filepath',
 'legacy_crc32_hash',
 'legacy_crc32c_hash',
 'legacy_size',
 'low_quality',
 'matched_normal',
 'md5_hash',
 'mediatype',
 'origin',
 'parent_cell_line',
 'participant_id',
 'primary_disease',
 'primary_site',
 'processing_qc',
 'release_date',
 'sequencing_date',
 'sequencing_type',
 'sex',
 'size',
 'sm_id',
 'source',
 'stripped_cell_name',
 'subsubtype',
 'subtype',
 'update_time',
 'version'}

In [114]:
ccle_refsamples[ccle_refsamples.source.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-6Je1tY,ACH-001605,2,NaN,NaN,rna,4.654329e+09,NaN,NO10,PT-oZhLPsoJ,CVCL_3075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-57hTZY,ACH-001046,1,NaN,NaN,rna,7.399863e+09,NaN,CORL26,PT-Z2Fq3yte,CVCL_2410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-j3NCLS,ACH-002047,1,NaN,NaN,rna,4.500937e+09,NaN,HIRSBM,PT-OOZstzSA,CVCL_E060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-FMkkVj,ACH-001648,1,NaN,NaN,rna,4.050264e+09,NaN,SHMAC4,PT-oBrDoJPm,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-6HLVzC,ACH-001227,1,NaN,NaN,rna,5.268985e+09,NaN,UPCISCC090,PT-mkNL4JuM,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-PoGjEx,ACH-001641,1,NaN,PDO-21298,rna,7.813216e+09,NaN,SAT,PT-uOqhQJIC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-J80Eb1,ACH-002034,1,NaN,PDO-21298,rna,4.940856e+09,NaN,T3M11,PT-hYmRJ4Oj,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-Hb7hHv,ACH-002080,1,NaN,PDO-21298,rna,4.613839e+09,NaN,TN2,PT-h2eeF6an,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-3VSkPT,ACH-002687,1,NaN,PDO-21569,rna,5.219565e+09,NaN,WM3772F,PT-MJutykcs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN


In [111]:
ccle_refsamples[ccle_refsamples.sex.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-yYeDem,ACH-001429,2,NaN,PDO-21569,rna,6.136561e+09,NaN,CCLFPEDS0009T,PT-gpiijlSj,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-OT217E,ACH-001338,1,NaN,PDO-21298,rna,5.395430e+09,NaN,CHP134,PT-sqRdd0QF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-a8BCnW,ACH-001996,1,NaN,PDO-21298,rna,5.692887e+09,NaN,ICC18,PT-fZWks7bl,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-zeFugQ,ACH-002707,1,NaN,PDO-21569,rna,4.253798e+09,NaN,IMSM2,PT-KQHkzAJs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-GOI63U,ACH-002077,1,NaN,PDO-21298,rna,4.736551e+09,NaN,LU165,PT-3za4R2YQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-ldvG6N,ACH-001553,1,NaN,PDO-21298,rna,4.695594e+09,NaN,MEGA2,PT-SMrpbPt7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-LFK9QA,ACH-002013,1,NaN,PDO-21298,rna,4.680192e+09,NaN,MM28,PT-ws3gMJ8I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-SdYhEt,ACH-002084,1,NaN,PDO-21298,rna,5.914711e+09,NaN,MMAC,PT-ViFHoQt0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-gyyVs5,ACH-001992,1,NaN,PDO-21298,rna,6.628526e+09,NaN,ONE58,PT-77bcwWNt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-IgYH22,ACH-001633,1,NaN,PDO-21298,rna,5.966996e+09,NaN,PEO6,PT-kjmIMuN4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN


In [261]:
wm.disable_hound().update_sample_attributes(a)

Successfully updated attributes '['legacy_bam_filepath', 'legacy_bai_filepath', 'legacy_size', 'legacy_crc32c_hash', 'internal_bam_filepath', 'internal_bam_filepath', 'size', 'crc32c_hash', 'md5_hash']' for 412 samples.


In [262]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq')

In [490]:
a = wm.get_samples()

In [300]:
ss= h.dups(ccle_refsamples['size'].tolist()+ccle_refsamples['legacy_size'].tolist())

In [307]:
issue = {val:[] for val in ss}
recom = []
for val in ss:
    for k,v in ccle_refsamples[(ccle_refsamples['size']==val)].iterrows():
        if v.size!= gcp.extractSize(gcp.lsFiles([v.internal_bam_filepath],"-l")[0]):
            recom.append(k)
        else:
            issue[val].append(k)
    for k,v in ccle_refsamples[(ccle_refsamples['legacy_size']==val)].iterrows():
        if v.legacy_size!= gcp.extractSize(gcp.lsFiles([v.legacy_bam_filepath],"-l")[0]):
            recom.append(k)    
        else:
            issue[val].append(k)

listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs


KeyboardInterrupt: 

In [422]:
e = gcp.lsFiles(a.legacy_bam_filepath.tolist(),"-L")

listing files in gs


In [418]:
wm.update_sample_attributes(a)

Successfully updated attributes '['legacy_bam_filepath', 'legacy_bai_filepath', 'legacy_size', 'legacy_crc32c_hash']' for 15 samples.
Hound executing batch upload of 150 records


In [321]:
for k, val in ccle_refsamples[(ccle_refsamples.datatype=='rna')&(~ccle_refsamples.internal_bam_filepath.str.contains("/rnasq_hg38",na=True))].internal_bam_filepath.iteritems():
    rn = "gs://cclebams/rna/"+'/'.join(val.split('/')[3:])
    print(rn)
    rni = rn[:-1]+'i'
    vali = val[:-1]+"i"
    #! gsutil cp $val $rn
    #! gsutil cp $vali $rni
    ccle_refsamples.loc[k, "internal_bam_filepath"] = rn
    ccle_refsamples.loc[k, "internal_bai_filepath"] = rni
    ccle_refsamples.loc[k, "size"] = gcp.extractSize(gcp.lsFiles([rn], '-l')[0],)[1]
    ccle_refsamples.loc[k, "crc32c_hash"] = gcp.extractHash(gcp.lsFiles([rn], '-L')[0])[1]
    ccle_refsamples.loc[k, "sequencing_date"]= seq.getBamDate([rn])[0]
    print(rn)
    break

gs://cclebams/rna/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam
Copying gs://cclebams/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam [Content-Type=application/octet-stream]...
/ [1 files][  4.8 GiB/  4.8 GiB]                                                
Operation completed over 1 objects/4.8 GiB.                                      
Copying gs://cclebams/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bai [Content-Type=application/octet-stream]...
/ [1 files][  6.5 MiB/  6.5 MiB]                                                
Operation completed over 1 objects/6.5 MiB.                                      
listing files in gs
listing files in gs
gs://cclebams/rna/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam
gs://cclebams/rna/rna/Getz_IBM_CellLine_RNAseq_June2020/RP-1561/RNA/CORL26/v1/CORL26.bam
Copying gs://cclebams/rna/Getz_IBM_CellLine_RNAseq_June2020

KeyboardInterrupt: 

In [377]:
ccle_refsamples.loc[a[a.legacy_bam_filepath.isna()].index].internal_bam_filepath

sample_id
CDS-2FgC1U    gs://cclebams/rnasq_hg38/CDS-2FgC1U.Aligned.so...
CDS-3VSkPT    gs://cclebams/rnasq_hg38/CDS-3VSkPT.Aligned.so...
CDS-57hTZY    gs://cclebams/rnasq_hg38/CDS-57hTZY.Aligned.so...
CDS-6HLVzC    gs://cclebams/rnasq_hg38/CDS-6HLVzC.Aligned.so...
CDS-6Je1tY    gs://cclebams/rnasq_hg38/CDS-6Je1tY.Aligned.so...
CDS-a8BCnW    gs://cclebams/rnasq_hg38/CDS-a8BCnW.Aligned.so...
CDS-bmH4HU    gs://cclebams/rnasq_hg38/CDS-bmH4HU.Aligned.so...
CDS-FMkkVj    gs://cclebams/rnasq_hg38/CDS-FMkkVj.Aligned.so...
CDS-GOI63U    gs://cclebams/rnasq_hg38/CDS-GOI63U.Aligned.so...
CDS-gyyVs5    gs://cclebams/rnasq_hg38/CDS-gyyVs5.Aligned.so...
CDS-Hb7hHv    gs://cclebams/rnasq_hg38/CDS-Hb7hHv.Aligned.so...
CDS-IgYH22    gs://cclebams/rnasq_hg38/CDS-IgYH22.Aligned.so...
CDS-j3NCLS    gs://cclebams/rnasq_hg38/CDS-j3NCLS.Aligned.so...
CDS-J80Eb1    gs://cclebams/rnasq_hg38/CDS-J80Eb1.Aligned.so...
CDS-ldvG6N    gs://cclebams/rnasq_hg38/CDS-ldvG6N.Aligned.so...
CDS-LFK9QA    gs://cclebams/rn

In [212]:
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'size']

CDS-6dPoX7    13488934939
CDS-zmiamL     7606219711
CDS-3wjudI     8571265034
CDS-uUP01E     8733556513
CDS-UFAuQl     8852095963
                 ...     
CDS-PoGjEx     7813216031
CDS-J80Eb1     4940855845
CDS-Hb7hHv     4613838702
CDS-3VSkPT     5219564814
CDS-2FgC1U     7056679221
Name: size, Length: 1516, dtype: int64

In [225]:
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'size'] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])]['internal_bam_filepath'].tolist(),'-l')]

ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'crc32c_hash'] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])]['internal_bam_filepath'].tolist(),'-L')]

listing files in gs
listing files in gs


In [156]:
ccle_refsamples[ccle_refsamples.datatype.isin(['wgs'])][["internal_bam_filepath", "internal_bai_filepath", "md5_hash","legacy_bam_filepath","legacy_bai_filepath",'legacy_size', 'legacy_crc32c_hash']]

,internal_bam_filepath,internal_bai_filepath,md5_hash,legacy_bam_filepath,legacy_bai_filepath,legacy_size,legacy_crc32c_hash
CDS-uhGMmZ,gs://cclebams/wgs_hg38/CDS-uhGMmZ.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-uhGMmZ.wgs_ccle.bai,6136551f350a8e8b6b2ca478ac951d49,gs://cclebams/wgs/G16444/G-401/v1/G-401.bam,gs://cclebams/wgs/G16444/G-401/v1/G-401.bai,1.834687e+11,puzdUg==
CDS-vNNLAr,gs://cclebams/wgs_hg38/CDS-vNNLAr.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-vNNLAr.wgs_ccle.bai,470c011136c545e5eea12cf71a2910c9,gs://cclebams/wgs/G32831/G-402/v2/G-402.bam,gs://cclebams/wgs/G32831/G-402/v2/G-402.bai,1.428883e+11,08KlNA==
CDS-KoNTMz,gs://cclebams/wgs_hg38/CDS-KoNTMz.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-KoNTMz.wgs_ccle.bai,9d00ef2ecef3854d16d3693733760507,gs://cclebams/wgs/G16449/CHP-212/v2/CHP-212.bam,gs://cclebams/wgs/G16449/CHP-212/v2/CHP-212.bai,1.651190e+11,NmAKsw==
CDS-Wmm2y2,gs://cclebams/wgs_hg38/CDS-Wmm2y2.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-Wmm2y2.wgs_ccle.bai,f81251027a48bf0dd030260c1b2d7ebf,gs://cclebams/wgs/G32831/KYM-1/v2/KYM-1.bam,gs://cclebams/wgs/G32831/KYM-1/v2/KYM-1.bai,1.860679e+11,nQhYUA==
CDS-IKutdC,gs://cclebams/wgs_hg38/CDS-IKutdC.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-IKutdC.wgs_ccle.bai,de1720234eaa10b3ee38fa69cc4f0649,gs://cclebams/wgs/G32831/A-204/v3/A-204.bam,gs://cclebams/wgs/G32831/A-204/v3/A-204.bai,1.513434e+11,c1qSGg==
...,...,...,...,...,...,...,...
CDS-lEoO4A,gs://cclebams/wgs/CDS-lEoO4A.bam,gs://cclebams/wgs/CDS-lEoO4A.bai,c5368347afae1031645835742185b799,gs://cclebams/wgs/CDS-lEoO4A.bam,gs://cclebams/wgs/CDS-lEoO4A.bai,8.854451e+10,tFgKDw==
CDS-lZD981,gs://cclebams/wgs/CDS-lZD981.bam,gs://cclebams/wgs/CDS-lZD981.bai,35e4b81c789393791ad5c1e2c70cfd1b,gs://cclebams/wgs/CDS-lZD981.bam,gs://cclebams/wgs/CDS-lZD981.bai,7.308927e+10,sbmvbA==
CDS-M9kJ4N,gs://cclebams/wgs/CDS-M9kJ4N.bam,gs://cclebams/wgs/CDS-M9kJ4N.bai,eb51988decdf6e53473006d833d4fce0,gs://cclebams/wgs/CDS-M9kJ4N.bam,gs://cclebams/wgs/CDS-M9kJ4N.bai,7.108076e+10,LdAyfw==
CDS-bNHw6E,gs://cclebams/wgs/CDS-bNHw6E.bam,gs://cclebams/wgs/CDS-bNHw6E.bai,c1cb0522c5ba22df53e1b684e6f12a64,gs://cclebams/wgs/CDS-bNHw6E.bam,gs://cclebams/wgs/CDS-bNHw6E.bai,7.505758e+10,NHvcFg==


## Check that we have all the cell lines we expect for this release

This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

_As the list cannot be parsed, we are not comparing it for now_

In [ ]:
# this function may not work - it hasn't been tested
url = 'https://docs.google.com/spreadsheets/d/1qus-9TKzqzwUMNWp8S1QP4s4-3SsMo2vuQRZrNXf7ag/edit?ts=5db85e27#gid=0&fvid=1627883727'

compareToCuratedGS(url, sample = newsample[0], samplesetname = samplesetname, colname = 'CN New to internal')

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 5 functions To generate the copy number dataset:

*   **BamToUnmappedRGBams_MC** vdauwera/BamToUnmappedRGBamsSnapshot ID: 3
*   **Generate_uBAM_File_List** gkugener/ArrayOfFilesToTxtSnapshot ID: 1
*   **Realign_WES_GATK4** gatk/PreProcessingForVariantDiscovery_GATK4Snapshot ID: 7
*   **CNV_sample_XX** gatk/CNV_Somatic_Pair_WorkflowSnapshot ID: 9
*   **Aggregate_CN_seg_files** gkugener/Aggregate_CN_seg_filesSnapshot ID: 2

This output file for download will be saved under the sample set under the combined_seg_file attribute.

There are several other tasks in this workspace. In brief:

*   **CNV_Somatic_Panel_Workflow_Agilent_XX** gatk/CNV_Somatic_Panel_WorkflowSnapshot ID: 11. This task was used in this workspace to generate the Sanger PON. In the Sanger dataset, there is a set of 40 normal cell lines samples (cell lines derived from matched normal tissue). We can use these to generate a PON to normalize to rather than using the Agilent PON we use for the other CCLE cell lines. This leads to less noisy results. HOWEVER, results using the PON from this workflow should not use the X chromosome, as the sanger normals are not exclusively female or male (it is likely a mix).
*   **SANGER_PON_CNV_sample_XX** gatk/CNV_Somatic_Pair_WorkflowSnapshot ID: 9. Same as the CNV_sample_XX_gatk, except that is uses the Sanger based PON. Should be used only for the Sanger cell lines.
*   **Sanger_PON_Aggregate_CN_seg_files** gkugener/Aggregate_CN_seg_filesSnapshot ID: 2. Aggregates the segment files for the samples that were run using the Sanger PON based CNV workflow.

### cleaning workspaces

In [ ]:
torm = terra.listHeavyFiles(refworkspace)
h.parrun(['gsutil rm '+i for i in torm], cores=8)
terra.removeFromFailedWorkflows(refworkspace, dryrun=False, everythingFor=['Realign_WES_GATK4','Generate_uBAM_File_List','BamToUnmappedRGBams_MC','CGA_WES_CCLE_ICE','CGA_WES_CCLE_AGILENT'])

## On Terra

In [ ]:
# a list of Terra workflows that are in the workspace and that we will call sequentially
bamtoubam= "BamToUnmappedRGBams_MC"
ubamtofilelist = "Generate_uBAM_File_List"
realign="Realign_WES_GATK4"

In [ ]:
# see dalmatian
subid = refwm.create_submission(bamtoubam,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
subid = refwm.create_submission(ubamtofilelist,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
subid = refwm.create_submission(realign,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
# Testing out the XY PoN for CN characterization. Will test by producing an output in a different column from usual so it's easy to delete the column attribute later
# Also, need to make a split between Agilent and ICE samples..
submission_id= refwm.create_submission("CNV_sample_XY_ice",etype='sample_set',entity=samplesetname, expression='this.samples')
terra.waitForSubmission(refworkspace,submission_id)

### copy pairs data to sample data

In [ ]:
pairs = refwm.get_pairs()

In [ ]:
pairs = pairs[~pairs['called_copy_ratio_segments_tumor'].isna()]
pairs = pairs.drop(columns=['case_sample','control_sample','participant'])
pairs.index = [i.split('_')[0] for i in pairs.index]

In [ ]:
refwm.update_sample_attributes(pairs)

continuing

In [ ]:
submission_id = refwm.create_submission("Aggregate_CN_seg_files",entity="all")
terra.waitForSubmission(refworkspace,submission_id)

__we are getting the results file path__

In [ ]:
terra.waitForSubmission(refworkspace,submission_id)
aggregated = refwm.get_entities('sample_set').loc['all']["combined_seg_file"]
aggregated

## On local

__We then save the workflow configurations used__

In [ ]:
terra.saveConfigs(refworkspace,'data/'+samplesetname+'/CNVconfig')

__delete unmapped bams generated during the process__

In [ ]:
toremove = ["readgroup_ubams",]
res = refwm.get_samples()
for val in toremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val], delete_files=True)

In [ ]:
# sometimes the previous step does not work and you need to do it manually (you can run this to check it worked)
for val in samplesinset.readgroup_ubams:
    ubams = ''
    if not type(val) is list:
        continue 
    for v in val:
        ubams+=' '+v
    os.system('gsutil -m rm'+ubams)

__and move the hg38 aligned bams to our own datastorage bucket__

Note that we may encounter some WGS files, which need to go to a different folder from the WES bam files.

In [ ]:
samplesetname

In [ ]:
#samplesinset = samples.index.tolist()
samplesinset= [i['entityName'] for i in refwm.get_entities('sample_set').loc[samplesetname].samples]
samplesinset

In [ ]:
onlycol = ['hg38_analysis_ready_bam', 'hg38_analysis_ready_bam_index', 'hg38_analysis_ready_bam_md5']
wes_newgs = 'gs://cclebams/hg38_wes/'
wes_res, flagged = terra.changeGSlocation(refworkspace, newgs=wes_newgs, onlysamples=samplesinset, onlycol=onlycol, entity='sample', keeppath=False, dry_run = False)

#### set it this way in our sample tracker

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
ccle_refsamples.loc[samplesinset,['legacy_bam_filepath','legacy_bai_filepath', 'legacy_size', 'legacy_crc32c_hash']] = ccle_refsamples.loc[samplesinset][['internal_bam_filepath', 'internal_bai_filepath', 'size', 'crc32c_hash']].values

ccle_refsamples.loc[samplesinset,'internal_bam_filepath'] = wes_res['hg38_analysis_ready_bam'].values

ccle_refsamples.loc[samplesinset,'internal_bai_filepath'] = wes_res['hg38_analysis_ready_bam_index'].values

ccle_refsamples.loc[wes_res.index.tolist(),'size'] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(wes_res['hg38_analysis_ready_bam'].tolist(),'-l')]

ccle_refsamples.loc[wes_res.index.tolist(),'crc32c_hash'] = [gcp.extractHash(i) for i in gcp.lsFiles(wes_res['hg38_analysis_ready_bam'].tolist(),'-L')]

ccle_refsamples.loc[wes_res.index.tolist(),'md5_hash'] = gcp.catFiles(wes_res['hg38_analysis_ready_bam_md5'].tolist(), cut=32)

In [121]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

### Get QC files

In [ ]:
dataBam = getQC(workspace=refworkspace ,only=samplesinset, qcname=["hg38_duplication_metrics","hg38_bqsr_report"])
dataCN = getQC(workspace=refworkspace ,only=samplesinset, qcname=["allelic_counts_tumor","delta_MAD_tumor","denoised_MAD_tumor","scaled_delta_MAD_tumor","denoised_copy_ratios_lim_4_plot_tumor","denoised_copy_ratios_plot_tumor","modeled_segments_plot_tumor"])

In [ ]:
for k,v in dataCN.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'processing_qc'] = str(v)
for k,v in dataBam.items():
    if k =='nan':
        continue
    ccle_refsamples.loc[k,'bam_qc'] = str(v)

In [ ]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

__We download and reprocess removing the appended version and keeping only the newest versions__

In [ ]:
! gsutil cp $aggregated "temp/cnv_ccle.called.seg"

In [ ]:
segments = pd.read_csv("temp/cnv_ccle.called.seg", sep='\t')
# TODO: copy allelic calls as well

In [ ]:
len(set(segments.Sample))

In [ ]:
segments = segments.rename(columns={'CONTIG':'Chromosome',
'START':'Start',
'END':'End',
'Sample':"DepMap_ID",
'NUM_POINTS_COPY_RATIO':'Num_Probes',
'MEAN_LOG2_COPY_RATIO':'Segment_Mean',
'CALL':'Status'})

In [ ]:
wrongswes = {'CDS-8Ut3sT',
 'CDS-Ip02tY',
 'CDS-Rd4nMx',
 'CDS-TGTiB8',
 'CDS-VnMBYD',
 'CDS-YSRYLi',
 'CDS-ZJh6UN',
 'CDS-dgxjAa',
 'CDS-0aJ4Yh',
 'CDS-0lfqVz',
 'CDS-0pZb0j',
 'CDS-1b1Hxk',
 'CDS-1djAlo',
 'CDS-1p2nnc',
 'CDS-34hKv3',
 'CDS-3EBt51',
 'CDS-3M6Pq9',
 'CDS-3WygAj',
 'CDS-49azaP',
 'CDS-4sr6RL',
 'CDS-5rD8XC',
 'CDS-5wYxZS',
 'CDS-6Yy3Yj',
 'CDS-6da3hu',
 'CDS-6l3V79',
 'CDS-7PFldq',
 'CDS-9XPgHB',
 'CDS-9qDPiX',
 'CDS-B0qAaq',
 'CDS-CMenCH',
 'CDS-CuJ0f8',
 'CDS-Dkl8OF',
 'CDS-Eq9UNX',
 'CDS-FRxdcH',
 'CDS-HNytLD',
 'CDS-Hj3xAa',
 'CDS-IJnjkY',
 'CDS-Ig6N9S',
 'CDS-KQDgIV',
 'CDS-KYkMDa',
 'CDS-KbbgMb',
 'CDS-KgRznV',
 'CDS-L0pDPl',
 'CDS-M8xDMS',
 'CDS-MLJbT2',
 'CDS-MnF3x8',
 'CDS-OCkOqy',
 'CDS-ODmXrP',
 'CDS-OgPf0h',
 'CDS-PYw8ID',
 'CDS-PdUZxY',
 'CDS-QHp4h4',
 'CDS-QU7ftt',
 'CDS-QVhVDT',
 'CDS-QXBhht',
 'CDS-SJq3p4',
 'CDS-Sp18uD',
 'CDS-TpDBjm',
 'CDS-TyWjJs',
 'CDS-UV1pVE',
 'CDS-UnDaBI',
 'CDS-UtrDTK',
 'CDS-W80jkV',
 'CDS-WedVJA',
 'CDS-WfjTcJ',
 'CDS-X3c4UY',
 'CDS-XQkXf4',
 'CDS-XevQNc',
 'CDS-YMIv9D',
 'CDS-YYLKZ0',
 'CDS-agZcmk',
 'CDS-bntBUl',
 'CDS-cAEii6',
 'CDS-cYWYp7',
 'CDS-cyuMYb',
 'CDS-d18Xie',
 'CDS-dpub1O',
 'CDS-eUqT7L',
 'CDS-eowEZF',
 'CDS-fXMRF9',
 'CDS-gIMBax',
 'CDS-gRA4SM',
 'CDS-iEULQm',
 'CDS-ihI7Dp',
 'CDS-iqPqOr',
 'CDS-jqOvtj',
 'CDS-kxNZ5S',
 'CDS-leGxSD',
 'CDS-nby0QM',
 'CDS-no7ysz',
 'CDS-o4dXGr',
 'CDS-oHu1Ik',
 'CDS-pXMN9C',
 'CDS-picEuX',
 'CDS-qZsCuJ',
 'CDS-rLRUbG',
 'CDS-txTRwz',
 'CDS-uQ8nnX',
 'CDS-yPSmxb',
'CDS-0qPmaJ',
 'CDS-1PXzlf',
 'CDS-1uWUTi',
 'CDS-294bk6',
 'CDS-2JxT1P',
 'CDS-2LFZYm',
 'CDS-2Q2Kia',
 'CDS-2hGt1N',
 'CDS-2lAFkD',
 'CDS-2xSJmZ',
 'CDS-3DHwSX',
 'CDS-3FueNQ',
 'CDS-3VNhFC',
 'CDS-3jIdRa',
 'CDS-3mvYnW',
 'CDS-3pZIvU',
 'CDS-49xzNU',
 'CDS-4BrJr7',
 'CDS-4S6juQ',
 'CDS-4ZOQQF',
 'CDS-4l9BUT',
 'CDS-5H2go6',
 'CDS-5IcijG',
 'CDS-5LNjjI',
 'CDS-5PXB9Y',
 'CDS-5ViPeM',
 'CDS-5bQzF2',
 'CDS-5hbofu',
 'CDS-6EyvRQ',
 'CDS-6Fc0S5',
 'CDS-6PZKz8',
 'CDS-6mq2Or',
 'CDS-6xyqy9',
 'CDS-75psAH',
 'CDS-7JWzyA',
 'CDS-7nEZFG',
 'CDS-7rcFYn',
 'CDS-83LhEq',
 'CDS-8aHSii',
 'CDS-8mpXJa',
 'CDS-8sQWae',
 'CDS-8yHnJv',
 'CDS-8z476r',
 'CDS-96DdrP',
 'CDS-9JpX07',
 'CDS-9M8GNS',
 'CDS-9sg0Pm',
 'CDS-9u5DMn',
 'CDS-9zidMf',
 'CDS-AJMYsd',
 'CDS-AOWMF3',
 'CDS-AjRIMt',
 'CDS-Awmxa5',
 'CDS-BRxHbu',
 'CDS-BnszE4',
 'CDS-Bojgi7',
 'CDS-C3hSav',
 'CDS-C7o0op',
 'CDS-CRPZeK',
 'CDS-CZstO2',
 'CDS-D6mIfI',
 'CDS-DIckeT',
 'CDS-DZMoWW',
 'CDS-Eh7ost',
 'CDS-Eo5oAR',
 'CDS-EpURcL',
 'CDS-EzZEgz',
 'CDS-Fz0HXE',
 'CDS-G1sVsw',
 'CDS-GINQfy',
 'CDS-GnBdHN',
 'CDS-H1oKTL',
 'CDS-H4hPhD',
 'CDS-HEoDm7',
 'CDS-HOVBCg',
 'CDS-HjGCvC',
 'CDS-HkZUmY',
 'CDS-HoW111',
 'CDS-Hv0i3y',
 'CDS-Hw6KuA',
 'CDS-Hx6zuD',
 'CDS-I7bMcd',
 'CDS-I97Uzq',
 'CDS-IGOgCK',
 'CDS-Iu8c04',
 'CDS-IzeN7a',
 'CDS-J3jfZW',
 'CDS-J6kDsZ',
 'CDS-JMfP1M',
 'CDS-JvOeJK',
 'CDS-K2tTmq',
 'CDS-Kswf83',
 'CDS-LCfY0q',
 'CDS-LNTGnh',
 'CDS-LOW19e',
 'CDS-LUm1Vn',
 'CDS-LVeuLY',
 'CDS-LifesX',
 'CDS-LnV7QY',
 'CDS-M1sAGX',
 'CDS-M8aV3P',
 'CDS-MOOIHL',
 'CDS-Md89va',
 'CDS-MhXQX3',
 'CDS-N83rwD',
 'CDS-NBnCDl',
 'CDS-NPG23x',
 'CDS-NXnWiI',
 'CDS-NZsio7',
 'CDS-NjunRu',
 'CDS-O1ShTQ',
 'CDS-O8dfj7',
 'CDS-OLgoE4',
 'CDS-OWJaXi',
 'CDS-OjLMVy',
 'CDS-OnIxUL',
 'CDS-OxQgBw',
 'CDS-P79y6z',
 'CDS-PHI8VT',
 'CDS-PYWxsh',
 'CDS-Pkk9e2',
 'CDS-Pku96X',
 'CDS-PyELSk',
 'CDS-QE7bdY',
 'CDS-Qbfoau',
 'CDS-Ql8GJZ',
 'CDS-QtTdY6',
 'CDS-QxeMJW',
 'CDS-R3txwY',
 'CDS-R6ehaT',
 'CDS-RFBAY6',
 'CDS-RWYJ02',
 'CDS-RnsUHX',
 'CDS-RxQhcq',
 'CDS-SO3AhH',
 'CDS-SvzhGj',
 'CDS-T10Uph',
 'CDS-TCqSJW',
 'CDS-TDblpN',
 'CDS-TSDUCK',
 'CDS-Twv1kD',
 'CDS-Ty3mgt',
 'CDS-UL1jLm',
 'CDS-UVxUrF',
 'CDS-UfC2Dz',
 'CDS-Uru0Mh',
 'CDS-UvBswk',
 'CDS-UxKEaK',
 'CDS-V2ZEuP',
 'CDS-V6Kk5q',
 'CDS-VBr00g',
 'CDS-VCuHjJ',
 'CDS-WAPQGk',
 'CDS-WHZolj',
 'CDS-WP95Oi',
 'CDS-Ww1LC7',
 'CDS-XJDBDj',
 'CDS-Xgu4mi',
 'CDS-XqaEOX',
 'CDS-Y27yfi',
 'CDS-YYd4ww',
 'CDS-YnodyM',
 'CDS-ZGlgTf',
 'CDS-ZMsoXe',
 'CDS-aDUHcI',
 'CDS-aGMcvr',
 'CDS-aXqwpM',
 'CDS-allHxr',
 'CDS-awunD8',
 'CDS-b9sdh9',
 'CDS-bPT1F0',
 'CDS-bdb5iE',
 'CDS-bons31',
 'CDS-c2Sowd',
 'CDS-cBOy2Z',
 'CDS-cKMeDY',
 'CDS-cMvnjL',
 'CDS-ck9vpG',
 'CDS-cmV75B',
 'CDS-ctVpqU',
 'CDS-dJqQ4g',
 'CDS-dNVjOc',
 'CDS-dPlJzz',
 'CDS-dWHWU3',
 'CDS-eGQYXr',
 'CDS-eZg4P8',
 'CDS-fLsYaB',
 'CDS-fRpNQH',
 'CDS-frzvLf',
 'CDS-fs8moU',
 'CDS-g0KUGN',
 'CDS-gCSYjV',
 'CDS-gKIdjs',
 'CDS-gsqqAz',
 'CDS-h4mOdz',
 'CDS-hOI086',
 'CDS-iKXYuH',
 'CDS-iRstNJ',
 'CDS-iX8vqU',
 'CDS-ik526H',
 'CDS-jHqXGP',
 'CDS-kAARUi',
 'CDS-kFiHZk',
 'CDS-kt2Gne',
 'CDS-ktRRkc',
 'CDS-l1OClV',
 'CDS-lSpYo6',
 'CDS-lTogDX',
 'CDS-ldrQm3',
 'CDS-leyYAD',
 'CDS-loy9vi',
 'CDS-m49nRz',
 'CDS-mGHY2S',
 'CDS-mazUYU',
 'CDS-mtMTts',
 'CDS-n7Fqfe',
 'CDS-nOKbmw',
 'CDS-nTW67d',
 'CDS-nYIBWR',
 'CDS-ocw0rP',
 'CDS-ogUnWk',
 'CDS-ohjYlg',
 'CDS-opnGD7',
 'CDS-qIc5x3',
 'CDS-qP2MBQ',
 'CDS-qUtkjN',
 'CDS-qaOoHQ',
 'CDS-qeIIoY',
 'CDS-qv2bpJ',
 'CDS-r5Ym7C',
 'CDS-rLadW7',
 'CDS-rQIdNN',
 'CDS-rQMY3G',
 'CDS-rUs3FP',
 'CDS-rVAuin',
 'CDS-ragHOy',
 'CDS-s7pOQR',
 'CDS-sCWLGL',
 'CDS-sbwn0P',
 'CDS-sieIuO',
 'CDS-soTPPi',
 'CDS-tORJC8',
 'CDS-tPR3fn',
 'CDS-tYXity',
 'CDS-tgnRyK',
 'CDS-u1AlUI',
 'CDS-uGZguG',
 'CDS-w7i5l7',
 'CDS-w8wJvh',
 'CDS-wSV3OM',
 'CDS-wWwBMZ',
 'CDS-wbPtTZ',
 'CDS-wlTAAF',
 'CDS-wpXVQk',
 'CDS-x21VqU',
 'CDS-x7srFK',
 'CDS-xCyamv',
 'CDS-xI8ZAZ',
 'CDS-xIv1KJ',
 'CDS-xKNh7Q',
 'CDS-yCSYHi',
 'CDS-ycD9px',
 'CDS-ydPJEM',
 'CDS-z8Bvmk',
 'CDS-ziEOXJ',
 'CDS-zwAn7G'}
%store wrongswes

In [ ]:
segments = segments[~segments.DepMap_ID.isin(wrongswes)]

### Priorization

add columns to seg file with arxspan ID, version. only keep the newest version for any given arxspan ID.
The process to keep the newest version of any given line is a little different from 20Q2 onwards, because don't have any dataset that uses the CDS-IDs for the data from 20Q1 or earlier.

We have to download the Taiga datasets from the previous quarter, see if we have any arxspan IDs with new data, and then replace with that data. We use the function called "removeOlderVersions" to do this.

In [ ]:
renaming = removeOlderVersions(names=set(segments.DepMap_ID.tolist()), refsamples=ccle_refsamples[ccle_refsamples.datatype=="wes"], arxspan_id="arxspan_id", version="version")

## post Procesing

The post processing happens in R using guillaume's functions, in brief:

- processSegments
- filterForCCLE
- interpolateGapsInSegmented
- extendEndsOfSegments
- reprioritizeData

In [ ]:
notWESnotlegacy = notWES - set(legacy_segments.DepMap_ID)
%store notWESnotlegacy
notWESnotlegacy

In [71]:
server = BiomartServer(ensemblserver)
ensmbl = server.datasets['hsapiens_gene_ensembl']
server.show_databases()

{'ENSEMBL_MART_ENSEMBL': Ensembl Genes 102,
 'ENSEMBL_MART_FUNCGEN': Ensembl Regulation 102,
 'ENSEMBL_MART_GENOMIC': Genomic features 102,
 'ENSEMBL_MART_MOUSE': Mouse strains 102,
 'ENSEMBL_MART_ONTOLOGY': Ontology,
 'ENSEMBL_MART_SEQUENCE': Sequence,
 'ENSEMBL_MART_SNP': Ensembl Variation 102}


In [64]:
server = BiomartServer( "http://www.nov2020.archive.ensembl.org/biomart" )
ensmbl = server.datasets['hsapiens_gene_ensembl']
server.show_databases()

genemapping = pd.read_csv(io.StringIO(ensmbl.search({
  'attributes': ['ensembl_gene_id','clone_based_ensembl_gene','hgnc_symbol','gene_biotype','entrezgene_id', 'start_position', 'end_position',"chromosome_name"]
}, header=1).content.decode()), sep='\t')
genemapping.columns = ['ensembl_gene_id','clone_based_ensembl_gene','hgnc_symbol','gene_biotype','entrezgene_id', 'start', 'end', 'Chromosome']
genemapping = genemapping[~(genemapping['clone_based_ensembl_gene'].isna() & genemapping['hgnc_symbol'].isna())]
genemapping.loc[genemapping[genemapping.hgnc_symbol.isna()].index,"hgnc_symbol"] = genemapping[genemapping.hgnc_symbol.isna()]['clone_based_ensembl_gene']
genemapping = genemapping.set_index("ensembl_gene_id")
genemapping['ensembl_id'] =  {k: i.hgnc_symbol+' ('+k+')' for k,i in genemapping.iterrows()}

In [69]:
genemapping[genemapping.gene_biotype=="protein_coding"]

,clone_based_ensembl_gene,hgnc_symbol,gene_biotype,entrezgene_id,start,end,chr
ensembl_gene_id,,,,,,,
ENSG00000198888,NaN,MT-ND1,protein_coding,4535.0,3307,4262,MT
ENSG00000198763,NaN,MT-ND2,protein_coding,4536.0,4470,5511,MT
ENSG00000198804,NaN,MT-CO1,protein_coding,4512.0,5904,7445,MT
ENSG00000198712,NaN,MT-CO2,protein_coding,4513.0,7586,8269,MT
ENSG00000228253,NaN,MT-ATP8,protein_coding,4509.0,8366,8572,MT
...,...,...,...,...,...,...,...
ENSG00000189181,NaN,OR14I1,protein_coding,401994.0,248681322,248682328,1
ENSG00000175137,NaN,SH3BP5L,protein_coding,80851.0,248810446,248825915,1
ENSG00000171161,NaN,ZNF672,protein_coding,79894.0,248838210,248849517,1


In [67]:
gene_rename

{'ENSG00000210049': 'MT-TF (ENSG00000210049)',
 'ENSG00000211459': 'MT-RNR1 (ENSG00000211459)',
 'ENSG00000210077': 'MT-TV (ENSG00000210077)',
 'ENSG00000210082': 'MT-RNR2 (ENSG00000210082)',
 'ENSG00000209082': 'MT-TL1 (ENSG00000209082)',
 'ENSG00000198888': 'MT-ND1 (ENSG00000198888)',
 'ENSG00000210100': 'MT-TI (ENSG00000210100)',
 'ENSG00000210107': 'MT-TQ (ENSG00000210107)',
 'ENSG00000210112': 'MT-TM (ENSG00000210112)',
 'ENSG00000198763': 'MT-ND2 (ENSG00000198763)',
 'ENSG00000210117': 'MT-TW (ENSG00000210117)',
 'ENSG00000210127': 'MT-TA (ENSG00000210127)',
 'ENSG00000210135': 'MT-TN (ENSG00000210135)',
 'ENSG00000210140': 'MT-TC (ENSG00000210140)',
 'ENSG00000210144': 'MT-TY (ENSG00000210144)',
 'ENSG00000198804': 'MT-CO1 (ENSG00000198804)',
 'ENSG00000210151': 'MT-TS1 (ENSG00000210151)',
 'ENSG00000210154': 'MT-TD (ENSG00000210154)',
 'ENSG00000198712': 'MT-CO2 (ENSG00000198712)',
 'ENSG00000210156': 'MT-TK (ENSG00000210156)',
 'ENSG00000228253': 'MT-ATP8 (ENSG00000228253)',
 

In [ ]:
segments['Chromosome'] = [i[3:] for i in segments['Chromosome'].tolist()]
# reverting logtransform of GTK
segments.Segment_Mean = 2**segments.Segment_Mean
segments.Start = segments.Start.astype(int)
segments.End = segments.End.astype(int)
# setting sex genes to half of their value for it to match relative concentration of other genes.
segments.loc[segments[segments.Chromosome.isin(['X','Y'])].index,'Segment_Mean'] = segments[segments.Chromosome.isin(['X','Y'])]['Segment_Mean']/2
segments = segments.sort_values(by=['DepMap_ID','Chromosome','Start','End'])
genemapping = genemapping.sort_values(by=['Chromosome','start','end'])

In [ ]:
# TODO: check on IGV maxvalue

## Validation step

Once the files are saved, we load them back in python and do some validations, in brief:

- mean,max,var...
- to previous version: same mean,max,var...
- checkAmountOfSegments: flag any samples with a very high number of segments
- checkGeneChangeAccrossAll: flag any genes which stay at a similar value across all samples

In [ ]:
len(segments[segments.Segment_Mean>1000])

In [ ]:
gapmergedsegs = manageGapsInSegments(segments)
genecn = toGeneMatrix(gapmergedsegs, gene_mapping)

In [ ]:
checkGeneChangeAccrossAll(genecn, thresh=0.025)

In [ ]:
genecn.values.min(), genecn.values.mean(), genecn.values.max()

In [ ]:
failed

In [ ]:
failed = checkAmountOfSegments(segments,thresh = 2000)

In [ ]:
# reparing QC when we have a better duplicate
ref=pd.DataFrame(ccle_refsamples[ccle_refsamples.datatype=="wes"]['arxspan_id'])
replace={}
for val in failed:
    if val in list(renaming.keys()):
        a = ref[ref.arxspan_id==ref.loc[val].arxspan_id].index
        for v in a:
            if v not in failed:
                replace.update({val:v})
                break
print(len(replace), len(failed))
for k, val in replace.items():
    renaming[val] = renaming.pop(k)

In [ ]:
wesfailed = set(failed) - set(replace.keys())
%store wesfailed
%store renaming

In [ ]:
%store -r wesfailed

In [ ]:
for i, (k, val) in enumerate(refwm.get_samples().loc[refwm.get_sample_sets().loc["all"].samples].iterrows()):
    if i>100:
        continue
    plot = val["modeled_segments_plot_tumor"]
    ! gsutil cp $plot temp/
    print(k)
    print(val['arxspan_id'], val['sex'])
    display(Image('temp/'+plot.split('/')[-1]))

These look bad in 20Q1: 
ACH-002511 (M140325), ACH-001370 (OCIP5X)

These CN plots subjectively appear to have too many segments in new 20Q2 samples: 
ACH-002399 (CDS-sukIAT, 21NT_1), ACH-002401 (CDS-tVy3GF, 21MT2_1), ACH-002400 (CDS-VUHMHG, 21MT1_1)

In [ ]:
prevgenecn = tc.get(name='internal-20q3-00d0', file='CCLE_gene_cn')

In [ ]:
# getting the previous versions to check that we have everything we should
#prevgenecn = tc.get(name='depmap-a0ab', file='CCLE_gene_cn')
prevgenecn = tc.get(name='internal-20q3-00d0', file='CCLE_gene_cn')

prevsegments = tc.get(name='depmap-a0ab', file='CCLE_segment_cn')
prev = set(prevgenecn.index.tolist())
prevgenecn.max().max()

### Comparison to replicates

### finding missmatch

In [ ]:
closest = findClosestMatching(genecn, CCLE_gene_cn, True)

In [ ]:
closest

In [ ]:
# for each replicats, if it is not what it is supposed to be, will print other replicates that exist for this cell lines, and print what it seems to be vs what it is supposed to be
issues = []
for k,v in closest.items():
    if ccle_refsamples.loc[k,'arxspan_id'] != v:
        print(k)
        print(ccle_refsamples[(ccle_refsamples.index!=k) & (ccle_refsamples.arxspan_id==v) & (ccle_refsamples.datatype=='wes')].index)
        print(v,ccle_refsamples.loc[k,'arxspan_id'])
        issues.append(k)

In [ ]:
issues

In [ ]:
match, corr =findClosestMatching(genecn, prevgenecn.loc[['ACH-000123',]], True, returncorr=True)

In [ ]:
ccle_refsamples = changeCellLineName(ccle_refsamples, datatype = "wes", dupdict={
'CDS-b5ElTm': "ACH-000157", 
"CDS-up4Vo5": "ACH-000662",
"CDS-CWA37D": "ACH-000825", 
"CDS-CCAK2f": "ACH-001328",
"CDS-2jBQ8n": "ACH-000757",
"CDS-T8W6P4": "ACH-000398",
"CDS-9TDVpH": "ACH-000685",
"CDS-dQKiht": "ACH-000375",
"CDS-Ckptje": "ACH-002291",
"CDS-ljFuDX": "ACH-001339",
"CDS-5x4qLj": "ACH-000608",
"CDS-UxJcOY": "ACH-000561",
"CDS-TUYedU": "ACH-000261",
"CDS-RLVrVE": "ACH-001523",
"CDS-6liik0": "ACH-000561",
"CDS-b5ElTm": "ACH-000157",
"CDS-u9hZ60": "ACH-000077",
"CDS-NUlX3d": "ACH-000458",
"CDS-2HO10g": "ACH-000278"})

In [ ]:
ccle_refsamples = cleanVersions(ccle_refsamples)

In [ ]:
iss=[]
for k,v in corr.iterrows():
    print(k, v.mean())
    try:
        if v[ccle_refsamples.loc[k,'arxspan_id']] < 0.75:
            print(v[[closest[k],ccle_refsamples.loc[k,'arxspan_id']]])
            continue
    except:
        a = np.argsort(v.values)[-5:]   
        if  v.values[a[-1]]>0.8:
            print(ccle_refsamples.loc[k,'arxspan_id'], corr.columns[a], v.values[a])
            continue
    iss.append(k)
issues = iss

In [ ]:
iss=[]
for val in issues:
    if val in renaming:
        v = ccle_refsamples.loc[val,'arxspan_id']
        a = ccle_refsamples[(ccle_refsamples.index!=val) & (ccle_refsamples.arxspan_id==v) & (ccle_refsamples.datatype=='wes')].index.tolist()
        a = [e for e in a if e not in issues]
        if len(a)>0:
            a[0] = renaming.pop(val)
        else:
            iss.append(val)
issues = iss

In [ ]:
%store issues
issues

In [ ]:
prevgenecn

### removing duplicates

In [ ]:
unmatched

In [ ]:
lis = {k: v for k,v in ccle_refsamples[ccle_refsamples.index.isin(set(genecn.index))]['arxspan_id'].iteritems()}

In [ ]:
ge = np.log2(1+genecn[cols])
ce = CCLE_gene_cn[cols]
prev = ce.index.tolist()
corr={val: {} for val in set(prev) & set(CCLE_gene_cn.index)}
for k,v in lis.items():
    if v in prev:
        corr[v][k] = spearmanr(ge.loc[k],ce.loc[v])[0]

In [ ]:
tomerge=[]
for k,v in corr.items():
    a = []
    for i in h.dups(v.values()):
        for l,w in v.items():
            if w == i:
                a.append(l)
    if len(a)>1:
        tomerge.append(a)
len(tomerge)

In [ ]:
segments[segments.DepMap_ID=="CDS-A7rsOJ"]

In [ ]:
rerenaming['ACH-002359']

In [ ]:
corr

In [ ]:
for k,v in corr.items():
    if v<0.7:
        print(k)
        print(set(priosegments[priosegments.DepMap_ID==k].Source), set(prevsegments[prevsegments.DepMap_ID==k].Source))

In [ ]:
tomerge

In [ ]:
wesdup= np.array(tomerge)
%store wesdup

In [ ]:
gcp.rmFiles(ccle_refsamples[ccle_refsamples.index.isin(np.array(tomerge)[:,1])][['internal_bam_filepath', 'internal_bai_filepath', 'legacy_bam_filepath', 'legacy_bai_filepath']].values.ravel())

In [ ]:
# TODO: replace in the renaming

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
ccle_refsamples = ccle_refsamples.drop(np.array(tomerge)[:,1])

In [ ]:
ccle_refsamples = cleanVersions(ccle_refsamples)

In [ ]:
#dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)
copyToWorkspace("broad-firecloud-ccle/DepMap_WES_CN_hg38", ccle_refsamples, deleteUnmatched=True)
copyToWorkspace("broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline", ccle_refsamples, deleteUnmatched=True)

### saving replicates

In [ ]:
wesfailed

In [ ]:
renaming

In [ ]:
segments = segments[~segments.DepMap_ID.isin(set(wesfailed) | set(wesdup[:,1]))]
genecn = genecn[~genecn.index.isin(set(wesfailed) | set(wesdup[:,1]))]

In [ ]:
for v in set(segments.DepMap_ID):
    segments.loc[segments[segments.DepMap_ID==v].index,'Source']= ccle_refsamples[ccle_refsamples.index==v].source.values[0]

In [ ]:
segments.Source = segments.Source.replace({'CCLF':'Broad WES', 'CHORDOMA':'Chordoma WES', 'SANGER':'Sanger WES', 'IBM':'Broad WES', np.nan:'Broad WES', 'DEPMAP':'Broad WES', 'IBM WES': "Broad WES", 'Broad CCLF':"Broad WES"})

In [ ]:
segments.to_csv('temp/segments_allWES_withreplicates_'+samplesetname+'.csv', index=False)
genecn.to_csv('temp/gene_cn_allWES_withreplicates_'+samplesetname+".csv")

In [ ]:
(set(renaming.keys()) - set(segments.DepMap_ID)) - wesfailed

In [ ]:
segments = pd.read_csv('temp/segments_allWES_withreplicates_'+samplesetname+'.csv')
genecn = pd.read_csv('temp/gene_cn_allWES_withreplicates_'+samplesetname+".csv", index_col=0)

### Comparison to prioritized

In [ ]:
# getting the other version if necessary, because the other on needs to be removed
for val in wesdup:
    if val[1] in renaming:
        renaming[val[0]] = renaming.pop(val[1])
        
# getting another version that did not fail
for val in wesfailed:
    v = ccle_refsamples[(ccle_refsamples.arxspan_id == ccle_refsamples.loc[val].arxspan_id[0]) & (ccle_refsamples.datatype=='wes')].index
    if len(v)>1:
        for k in v:
            if k != val:
                renaming[k] = renaming.pop(val)
    else:
        try:
            renaming.pop(val)
        except:
            print('already removed')
%store renaming

In [ ]:
set(["ACH-000274",
"ACH-002446",
"ACH-000833",
"ACH-001151",
"ACH-001955",
"ACH-000757",
"ACH-000511",
"ACH-001321",
"ACH-000473",
"ACH-001605",
"ACH-001957"]) - set(priosegments.DepMap_ID)

In [ ]:
renaming.update({"CDS-Ckptje": "ACH-001672",
"CDS-pgDmZb": "ACH-002291"})

In [ ]:
priosegments = segments[segments.DepMap_ID.isin(set(renaming.keys()))].replace(renaming)
priogenecn = genecn[genecn.index.isin(set(renaming.keys()))].rename(index=renaming)

In [ ]:
cols = set(priogenecn.columns) & set(CCLE_gene_cn.columns)
ge = np.log2(1+priogenecn[cols])
ce = CCLE_gene_cn[cols]
ind = set(priogenecn.index) & set(CCLE_gene_cn.index)
corr={}
for val in ind:
    corr[val] = pearsonr(ge.loc[val],ce.loc[val])[0]
for k,v in corr.items():
    if v<0.3:
        print(k)
        print(set(priosegments[priosegments.DepMap_ID==k].Source), set(prevsegments[prevsegments.DepMap_ID==k].Source))

In [ ]:
for k,v in corr.items():
    if v<0.3:
        print(k)
        print(set(priosegments[priosegments.DepMap_ID==k].Source), set(CCLE_segment_cn[CCLE_segment_cn.DepMap_ID==k].Source))

In [ ]:
a = np.array(list(corr.values()))
sns.kdeplot(a)

In [ ]:
sns.scatterplot(x=ge.loc[ind].values.ravel()[:100000],y=ce.loc[ind].values.ravel()[:100000],)

In [ ]:
sns.kdeplot(data=np.array([ge.loc[ind].values.ravel()[:100000], ce.loc[ind].values.ravel()[:100000]]).T, fill=True)

### saving prioritizd

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)
normals = ccle_refsamples[ccle_refsamples['primary_disease']=='normal'].index.tolist()
normals

In [ ]:
#priosegments = priosegments[~priosegments.DepMap_ID.isin(normals)]
#priogenecn = priogenecn.drop(index=normals)

In [ ]:
#h.compareDfs(priosegments, tc.get(name='depmap-a0ab', file='CCLE_segment_cn'))
h.compareDfs(priogenecn, tc.get(name='depmap-a0ab', file='CCLE_gene_cn'))

In [ ]:
priosegments.to_csv("temp/segments_allWES_latest_"+samplesetname+".csv", index=False)
priogenecn.to_csv('temp/gene_cn_allWES_latest_'+samplesetname+".csv")

In [ ]:
priosegments= pd.read_csv("temp/segments_allWES_latest_"+samplesetname+".csv")
priogenecn = pd.read_csv('temp/gene_cn_allWES_latest_'+samplesetname+".csv", index_col=0)

looking for correlation issues to previous releases

In [ ]:
a = set(priogenecn.columns) & set(prevgenecn.columns)

In [ ]:
corr = np.corrcoef(priogenecn)

In [ ]:
ind = priogenecn.index.tolist()

In [ ]:
a = [(ind[val[0]], ind[val[1]]) for val in np.argwhere(corr>0.96) if val[0]!=val[1]]

In [ ]:
len(a)

In [ ]:
['ACH-000131','ACH-000125','ACH-001093','ACH-000284','ACH-000340', 'ACH-000214', 'ACH-001767','ACH-000240','ACH-000154','ACH-000063','ACH-000165']

In [ ]:
[val for val in a if val[1] not in ['ACH-000131','ACH-000125','ACH-001093','ACH-000284','ACH-000340', 'ACH-000214', 'ACH-001767','ACH-000240','ACH-000154','ACH-000063','ACH-000165']]

### saving samples version used for the release

In [ ]:
ccle_refsamples.columns[-14:-1]

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)

In [ ]:
ccle_refsamples

In [ ]:
loc = set(wes_res.index) & set(ccle_refsamples.index)
for i in ccle_refsamples.columns[-14:-1]:
    ccle_refsamples.loc[loc,i] = 0
ccle_refsamples[samplesetname]=0
ccle_refsamples.loc[renaming.keys(),samplesetname]=1
ccle_refsamples.loc[failed,'low_quality']=1
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

# Upload to taiga

- we load the blacklisted/embargoed sample ids
- we log2 transform and create a file for each release (and one containing everything)
- we upload the files using taigapy in a corresponding taiga dataset with the corresponding description and also upload it to its virtual dataset

## we push full dataset version in depmap taiga CN

In [ ]:
tc.update_dataset(dataset_permaname="cn-latest-d8d4", 
                  upload_file_path_dict={
    'temp/segments_allWES_latest_'+samplesetname+'.csv': 'TableCSV',
    'temp/gene_cn_allWES_latest_'+samplesetname+".csv": 'NumericMatrixCSV',
    'temp/segments_allWES_withreplicates_'+samplesetname+'.csv': 'TableCSV',
    'temp/gene_cn_allWES_withreplicates_'+samplesetname+".csv": 'NumericMatrixCSV',
    'temp/gene_cn_all_merged_'+samplesetname+".csv":"NumericMatrixCSV",
    'temp/segments_allWGS_withreplicates_'+samplesetname+".csv":"TableCSV",
    'temp/gene_cn_allWGS_withreplicates_'+samplesetname+".csv":"NumericMatrixCSV"},
                  changes_description=
"""
""",
#"adding:"+len(new)+"lines and removed"+str(removed)+" and adding WGS data! with 29 new lines",
                  dataset_description="""
# Copy Number

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal.

/!\ This is the most up to date version of the CCLE CN data.

# Notations:

all: everything

allWES: all data comes from the WExomeS samples we posses

allWGS: all data comes from the WGenomeS samples we posses

withreplicates: if we have two different sequencing from a sample, we kept both, see the depmap sample tracker for annotations [https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE](https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE). this dataset is more geared toward QC or in-depth analysis of a particular cell line.

merged: everything from both WGS and WES

latest: only the latest sequencing versions of the samples were kept


Gene level CN data:

__Rows__: cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segment level data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Segment\_Mean, Num\_Probes, Calls""")